In [1]:
from delta. tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

# Create a Spark Session
spark = SparkSession.builder \
    .appName("KafkaReadTransformWriteToKafka") \
    .config("spark.jars", "/work/spark-sql-kafka-0-10_2.12-3.5.1.jar,/work/kafka-clients-3.7.0.jar,work/delta-core_2.12-2.4.0.jar") \
    .getOrCreate()

In [2]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, StructType

# Define the schema of your JSON data
schema = StructType([
    StructField("jsonrpc", StringType()),
    StructField("method", StringType()),
    StructField("params", StructType([
        StructField("channel", StringType()),
        StructField("data", StructType([
            StructField("funding_8h", DoubleType()),
            StructField("current_funding", DoubleType()),
            StructField("estimated_delivery_price", DoubleType()),
            StructField("best_bid_amount", DoubleType()),
            StructField("best_ask_amount", DoubleType()),
            StructField("best_bid_price", DoubleType()),
            StructField("best_ask_price", DoubleType()),
            StructField("interest_value", DoubleType()),
            StructField("open_interest", LongType()),
            StructField("max_price", DoubleType()),
            StructField("min_price", DoubleType()),
            StructField("last_price", DoubleType()),
            StructField("settlement_price", DoubleType()),
            StructField("instrument_name", StringType()),
            StructField("mark_price", DoubleType()),
            StructField("index_price", DoubleType()),
            StructField("stats", StructType([
                StructField("volume_notional", DoubleType()),
                StructField("volume_usd", DoubleType()),
                StructField("volume", DoubleType()),
                StructField("price_change", DoubleType()),
                StructField("low", DoubleType()),
                StructField("high", DoubleType()),
            ])),
            StructField("state", StringType()),
            StructField("timestamp", LongType()),
        ])),
    ])),
])

In [3]:
data_df=spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers","kafka:9092")\
  .option("subscribe","real_time_data_options")\
  .option("failOnDataLoss","False")\
  .option("startingOffsets", "latest")\
  .load() 
  # .selectExpr(schema)

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [4]:
data_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
data_df.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "\work\delta\cp1")\
    .trigger(processingTime="30 seconds")\
    .start("\work\delta\dtable1")


Py4JJavaError: An error occurred while calling o72.start.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:370)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:233)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.